In [ ]:
!unzip /content/dataset/ml-1m.zip

Archive:  /content/dataset/ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
#imports
import re
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [ ]:
movies_df = data = pd.read_csv('/content/ml-1m/movies.dat', delimiter='::', header=None, names=['ID', 'Title', 'Genres'], encoding='latin1')
users_df = data = pd.read_csv('/content/ml-1m/users.dat', delimiter='::', header=None, names=['ID', 'Gender', 'Age', 'Occupation', 'Zip-Code'], encoding='latin1')
ratings_df = data = pd.read_csv('/content/ml-1m/ratings.dat', delimiter='::', header=None, names=['UserId', 'MovieId', 'Rating', 'Timestamp' ], encoding='latin1')

<ipython-input-3-5ba5c73d5254>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df = data = pd.read_csv('/content/ml-1m/movies.dat', delimiter='::', header=None, names=['ID', 'Title', 'Genres'], encoding='latin1')
<ipython-input-3-5ba5c73d5254>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users_df = data = pd.read_csv('/content/ml-1m/users.dat', delimiter='::', header=None, names=['ID', 'Gender', 'Age', 'Occupation', 'Zip-Code'], encoding='latin1')
<ipython-input-3-5ba5c73d5254>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (sep

In [ ]:
movies_df.head()

,ID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**Merging the all the three entities**

In [ ]:
user_ratings_df = pd.merge(users_df, ratings_df, left_on=['ID'], right_on=['UserId'], how='inner')
user_ratings_df.drop(columns=['ID'], inplace=True)
movie_user_rating_df = pd.merge(user_ratings_df, movies_df, left_on=['MovieId'], right_on=['ID'], how='inner')
movie_user_rating_df.drop(columns=['ID','UserId', 'MovieId'], inplace=True)

**Splitting the data into test and train datasets**

In [ ]:
y = movie_user_rating_df['Rating']
X  = movie_user_rating_df.drop(columns=['Rating'])

In [ ]:
X.head()

,Gender,Age,Occupation,Zip-Code,Timestamp,Title,Genres
0,F,1,10,48067,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,M,56,16,70072,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,M,25,12,32793,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,M,25,7,22903,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,M,50,1,95350,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def split_genres(genres):
    return genres.split('|')

# Apply split_genres function to both train and test data
X_train['Genres'] = X_train['Genres'].apply(split_genres)
X_test['Genres'] = X_test['Genres'].apply(split_genres)

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Perform one-hot encoding on train and test data
train_encoded = pd.DataFrame(mlb.fit_transform(X_train['Genres']), columns=mlb.classes_, index=X_train.index)
test_encoded = pd.DataFrame(mlb.transform(X_test['Genres']), columns=mlb.classes_, index=X_test.index)

X_train = pd.concat([X_train, train_encoded], axis=1)
X_test = pd.concat([X_test, test_encoded], axis=1)
X_train.drop(columns=['Genres'], inplace=True)
X_test.drop(columns=['Genres'], inplace=True)

In [ ]:
# Encode the Zip-Code column using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X_train['Zip-Code'] = label_encoder.fit_transform(X_train['Zip-Code'])
X_test['Zip-Code'] = label_encoder.transform(X_test['Zip-Code'])


In [ ]:
# Encode the Gender column using LabelEncoder
X_train['Gender'] = label_encoder.fit_transform(X_train['Gender'])
X_test['Gender'] = label_encoder.transform(X_test['Gender'])


In [ ]:
X_train.drop(columns=['Title'], inplace=True)
X_test.drop(columns=['Title'], inplace=True)

In [ ]:
X_train.head()

,Gender,Age,Occupation,Zip-Code,Timestamp,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
416292,1,18,6,721,974942836,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
683230,1,25,14,2913,997321489,1,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2434,1,25,20,3190,974713988,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
688533,0,25,0,1788,960183537,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
472584,1,35,0,3295,965370901,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


**Performance using Gradient Boosting Classifier**

In [ ]:
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, precision_score, recall_score

# Create and train the HistGradientBoostingClassifier model
model = HistGradientBoostingClassifier(max_iter=100)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Calculate and display relevant metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
accuracy = model.score(X_test, y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # weighted averaging for multi-class
recall = recall_score(y_test, y_pred, average='weighted')

print(f"R^2: {r2:.3f}")
print(f"RMSE: {rmse:.3f}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")



R^2: -0.063
RMSE: 1.153
Accuracy: 0.386
Precision: 0.408
Recall: 0.386


**Performance using AdaBoostClassifier**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score


# Initialize the AdaBoostClassifier with LogisticRegression as the base model
clf = AdaBoostClassifier(n_estimators=7, random_state=42)

# Train the model on the training data
clf.fit(X_train, y_train)

# Predict on the test data
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # weighted averaging for multi-class
recall = recall_score(y_test, y_pred, average='weighted')

# Calculate root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print the result
print("Root mean squared error:", rmse)
print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")


Root mean squared error: 1.2001322982563227
Accuracy: 0.350
Precision: 0.281
Recall: 0.350


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Performance using Stacking Classifier containing the following:**

1.   Layer 1: Random Forest, Extra Trees, AdaBoost Classifier
2.   Layer 2: Historgram Gradient Boosted Trees Clasifier



In [ ]:
import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, HistGradientBoostingClassifier


# Define the base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42, min_samples_split=5)),
    ('extraTrees', ExtraTreesClassifier(n_estimators=10, random_state=42, min_samples_split=5)),
    ('ada', AdaBoostClassifier(n_estimators=7, random_state=42))
]

# Define the meta model
meta_model = HistGradientBoostingClassifier(max_iter=100)

# Create the StackingClassifier
clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Train the model on the training data
clf.fit(X_train, y_train)

# Predict on the test data
y_pred = clf.predict(X_test)

# Calculate and display relevant metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Accuracy: {accuracy:.3f}")
print(f"F1 Score: {f1:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"RMSE: {rmse:.3f}")



Accuracy: 0.395
F1 Score: 0.301
Precision: 0.396
Recall: 0.297
RMSE: 1.127
